In [5]:
import numpy as np
import pandas as pd
from numerical_methods.eigen_system import power_method, inverse_power_method, wielandt_deflation
import time
from returns.result import Result, Success, Failure

In [6]:
def rand_mat(n=50):
    B = np.random.normal(size=(n, n))
    A = np.eye(n) / 2 + B + B.T
    return A


def vec_err(A, x, val, val_true):
    return np.linalg.norm(A @ x - val * x, ord=np.inf) / np.abs(val_true)

In [7]:
pow_time = []
pow_val_err = []
pow_vec_err = []
inv_time = []
inv_val_err = []
inv_vec_err = []

def_pow_time = []
def_pow_val_err = []
def_pow_vec_err = []
def_inv_time = []
def_inv_val_err = []
def_inv_vec_err = []

for _ in range(25):
    A = rand_mat(n=50)
    x0 = np.ones(shape=(50,1))
    
    val_true, vec_true = np.linalg.eig(A)
    sorted_idx = np.argsort(np.abs(val_true))
    max_idx = sorted_idx[-1]
    min_idx = sorted_idx[0]
    snd_max_idx = sorted_idx[-2]
    snd_min_idx = sorted_idx[1]

    # Largest Eig solved by power method
    pow_start = time.time()
    val_pow, vec_pow = power_method(A, x0).unwrap()
    pow_time.append(time.time() - pow_start)
    pow_val_err.append(np.abs(val_pow - val_true[max_idx]))
    pow_vec_err.append(vec_err(A, vec_pow, val_pow, val_true[max_idx]))
    
    # Smallest Eig solved by inverse power method
    inv_start = time.time()
    val_inv, vec_inv = inverse_power_method(A, x0, q=0).unwrap()
    inv_time.append(time.time() - inv_start)
    inv_val_err.append(np.abs(val_inv - val_true[min_idx]))
    inv_vec_err.append(vec_err(A, vec_inv, val_inv, val_true[min_idx]))
    
    # Second Largest Eig solved by deflation method using power method as solver
    def_pow_start = time.time()
    x00 = x0[:-1]
    val_def_pow, vec_def_pow = wielandt_deflation(A, x00, val_pow, vec_pow, solver=power_method).unwrap()
    def_pow_time.append(time.time() - def_pow_start)
    def_pow_val_err.append(np.abs(val_def_pow - val_true[snd_max_idx]))
    def_pow_vec_err.append(vec_err(A, vec_def_pow, val_def_pow, val_true[snd_max_idx]))
    
    # Second Smallest Eig solved by deflation method using power method as solver
    def_inv_start = time.time()
    x00 = x0[:-1]
    val_def_inv, vec_def_inv = wielandt_deflation(A, x00, val_inv, vec_inv, solver=inverse_power_method).unwrap()
    def_inv_time.append(time.time() - def_inv_start)
    def_inv_val_err.append(np.abs(val_def_inv - val_true[snd_min_idx]))
    def_inv_vec_err.append(vec_err(A, vec_def_inv, val_def_inv, val_true[snd_min_idx]))

In [8]:
df = pd.DataFrame({
    "Smallest Eig": [np.mean(xs) for xs in [inv_time, inv_val_err, inv_vec_err]],
    "Largest Eig": [np.mean(xs) for xs in [pow_time, pow_val_err, pow_vec_err]], 
    "Second Smallest Eig": [np.mean(xs) for xs in [def_inv_time, def_inv_val_err, def_inv_vec_err]], 
    "Second Largest Eig": [np.mean(xs) for xs in [def_pow_time, def_pow_val_err, def_pow_vec_err]], 
}).T
df = df.set_axis(["lambda time", "lambda error", "Vector Error"], axis=1)
df

,lambda time,lambda error,Vector Error
Smallest Eig,0.005557,0.000006,0.000066
Largest Eig,0.008837,0.000426,0.000096
Second Largest Eig,0.012146,0.009372,0.011311
Second Smallest Eig,0.011730,0.000053,0.000057
